<a href="https://colab.research.google.com/github/GeomaticsCaminosUPM/SeismicBuildingExposure/blob/main/SeismicBuildingExposure/footprint/examples/example_irregularity_norms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Irregularity module example

This module computes footprint irregularity indices of buildings

Install libraries if using colab

In [1]:
!pip install geopandas
!pip install "SeismicBuildingExposure[footprint] @ git+https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure.git"
!pip install folium matplotlib mapclassify

  Cloning https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure.git to /tmp/pip-install-pzzd3h4b/seismicbuildingexposure_88e7753bd2764988b0ea622e9a8ffd2a
  Running command git clone --filter=blob:none --quiet https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure.git /tmp/pip-install-pzzd3h4b/seismicbuildingexposure_88e7753bd2764988b0ea622e9a8ffd2a
  Resolved https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure.git to commit 502a98f688fd661a890036ed69b930a5ab0ea023
  Preparing metadata (setup.py) ... done
  Created wheel for SeismicBuildingExposure: filename=SeismicBuildingExposure-0.1.0-py3-none-any.whl size=36449 sha256=b8cd7ba1b5853d9af86a4e23aedcf1a53d5aba24cbf955640c62759fb8a8d317
  Stored in directory: /tmp/pip-ephem-wheel-cache-hakccp64/wheels/77/97/30/347f7040b2fd480c655b72d69baeb6c5ee81c9bc5eaed6c257
Successfully built SeismicBuildingExposure


In [4]:
import SeismicBuildingExposure.footprint.shape
import geopandas as gpd

Documentation of every function can be found using `help()` or under https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure

In [5]:
help(SeismicBuildingExposure.footprint.shape.eurocode_8_df)

Help on function eurocode_8_df in module SeismicBuildingExposure.footprint.shape:

eurocode_8_df(geoms: geopandas.geodataframe.GeoDataFrame | geopandas.geoseries.GeoSeries) -> pandas.core.frame.DataFrame



## 1. Load data

Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

Download an example footprints file if needed

In [8]:
!curl -L -o footprints.gpkg https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure/raw/refs/heads/main/SeismicBuildingExposure/footprint/examples/footprints.gpkg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  364k  100  364k    0     0   656k      0 --:--:-- --:--:-- --:--:--  656k


Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

In [9]:
footprints_gdf = gpd.read_file('footprints.gpkg')

- All geometries should be single part and polygons.

    - Multi part geometries (MultiPolygon) would mean two buildings are contained in the same footprint geometry which should not happen.

    - Footprints must be polygons, not linestrings.

Check if all geometries are `Polygon`

In [10]:
if any(footprints_gdf.geometry.type == 'MultiPolygon'):
    print("There are multiplart geometries.")

if any(footprints_gdf.geometry.type.str.contains('Polygon') == False):
    print("Some geometries are not Polygon")

There are multiplart geometries.


Explode multipart geometries into single parts if needed


Note: The row number will be reset. Maybe you do not know how to match the footprints with your data. An id column is a good idea to solve this issue.

In [11]:
footprints_gdf = footprints_gdf.explode().reset_index() # The new index column is the row number in the origninal dataframe
footprints_gdf

,index,gid,geometry
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1..."
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1..."
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1..."
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1..."
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1..."
...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1..."
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1..."
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1..."
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1..."


## 2. Irregularity norms



### 2.1. Eurocode 8

In [12]:
eurocode_8 = SeismicBuildingExposure.footprint.shape.eurocode_8_df(footprints_gdf)
eurocode_8

,eccentricity_ratio,radius_ratio,slenderness,compactness,angle_eccentricity,angle_slenderness
0,0.114064,0.348005,1.232601,1.000000,7.401332,14.913346
1,0.004225,0.163555,3.192047,0.999820,73.967503,3.895212
2,0.015982,0.279843,2.961852,0.999880,-44.943716,24.531446
3,0.006233,0.232615,3.338191,0.997747,-37.764563,-85.457928
4,0.106776,0.223017,2.101109,0.835991,1.610841,-79.460449
...,...,...,...,...,...,...
939,0.336837,0.212981,1.100164,0.957760,-53.048613,-83.267213
940,0.005441,0.297199,1.158917,1.000000,-33.810798,-77.275465
941,0.068885,0.339464,1.739688,0.929391,-44.692859,13.023239
942,0.056498,0.438822,1.350890,0.882097,76.603246,27.300675


In [13]:
footprints_gdf['compactness_EC8'] = eurocode_8['compactness'] # Save desired columns in footprints_gdf

### 2.2. Costa Rica building code

In [14]:
costa_rica = SeismicBuildingExposure.footprint.shape.codigo_sismico_costa_rica_df(footprints_gdf)
costa_rica

,eccentricity_ratio,angle
0,0.017475,-81.520791
1,0.000651,-3.748371
2,0.006153,33.439118
3,0.001112,54.645441
4,0.023138,-84.707905
...,...,...
939,0.024799,36.706984
940,0.000682,56.123849
941,0.020170,41.327770
942,0.019684,-12.771424


In [16]:
footprints_gdf['eccentricity_ratio_CR'] = costa_rica['eccentricity_ratio'] # Save desired columns in footprints_gdf

### 2.3. Mexico NTC norm

In [18]:
mexico = SeismicBuildingExposure.footprint.shape.NTC_mexico_df(footprints_gdf)
mexico

,setback_ratio,hole_ratio
0,0.000000,0.0
1,0.006196,0.0
2,0.000230,0.0
3,0.004414,0.0
4,0.338729,0.0
...,...,...
939,0.209318,0.0
940,0.000000,0.0
941,0.282326,0.0
942,0.389056,0.0


In [19]:
footprints_gdf['setback_ratio_NTC'] = mexico['setback_ratio'] # Save desired columns in footprints_gdf
footprints_gdf['hole_ratio_NTC'] = mexico['hole_ratio'] # Save desired columns in footprints_gdf

## 2.4 US ASCE code

In [25]:
asce = SeismicBuildingExposure.footprint.shape.asce_7_df(footprints_gdf)
asce

,setback_ratio,hole_ratio,parallelity_angle
0,0.000000,0.0,12.297284
1,0.006196,0.0,0.334010
2,0.000230,0.0,2.943516
3,0.004414,0.0,0.542560
4,0.338729,0.0,0.527128
...,...,...,...
939,0.209318,0.0,2.370070
940,0.000000,0.0,1.100585
941,0.282326,0.0,1.629774
942,0.389056,0.0,1.290386


In [28]:
footprints_gdf['setback_ratio_ASCE'] = asce['setback_ratio'] # Save desired columns in footprints_gdf
footprints_gdf['hole_ratio_ASCE'] = asce['hole_ratio'] # Save desired columns in footprints_gdf
footprints_gdf['parallelity_angle_ASCE'] = asce['parallelity_angle'] # Save desired columns in footprints_gdf

## 3. Example code to plot results


Plot a map (needs pip install folium matplotlib mapclassify)

### 3.1. Compactness Eurocode 8

In [20]:
footprints_gdf.explore(
    column='compactness_EC8',       # Column to base the color on
    cmap='RdYlGn',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    vmin=0.8,
    vmax=1
)

### 3.2. Eccentricity ratio Costa Rica building code

In [22]:
footprints_gdf.explore(
    column='eccentricity_ratio_CR',       # Column to base the color on
    cmap='RdYlGn_r',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    vmin=0,
    vmax=0.15
)

### 3.3. Setback ratio Mexico NTC

In [23]:
footprints_gdf.explore(
    column='setback_ratio_NTC',       # Column to base the color on
    cmap='RdYlGn_r',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    vmin=0,
    vmax=0.4
)

## 3.4 ASCE parallelity

In [30]:
footprints_gdf.explore(
    column='parallelity_angle_ASCE',       # Column to base the color on
    cmap='RdYlGn_r',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    vmin=0,
    vmax=15
)

## 4. Save results

In [24]:
footprints_gdf.to_file('file.gpkg')